```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```

전체 구간에 대해 유전 알고리즘을 적용시킴니다.

In [73]:
# import 및 변수정의 구간
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

# 역 구간
Station_section = 11

# 하루에 운행하는 열차 수
One_day_train = 100

#교차율
Pc = 0.7

#변이율 역수 관계
Pm = 1000

#마지막 세대수 
largest_generation = 500

#보존할 상위값
save_value = 5


In [74]:
# 01010로 이루어진 리스트를 10진수로 디코딩하는 함수

def decoding(X, Y=0):
    """
    input   : 0100로 이루어진 리스트
    return : 10진수 정수
    """
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [75]:
#파일을 2차원 배열로 바꿔주기

def Readtxt(fname):
    """
    input  : 파일들
    return : 2차원 배열 
    """
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [82]:
# 바뀌야됨

def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return 777 - (6 * sp)

In [83]:
#초기값
speedlist = [[[[random.choice([0,1]) for i in range(listN)]for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]



In [84]:
#하루 열차의 총합 구하기
def Make_Score_per_all(speedlist):
    #열차마다 점수                             #30개의 경쟁 100개의 열차
    Score_per_train = [[0 for i in range(One_day_train)]for j in range(N)]

    # 30개의 경쟁
    for k in range(N):
        # 100대의 열차
        for i in range(One_day_train):
            # 11개의 역
            for j in range(Station_section):
                #각 열차에 대해      += 11개의 구간의 점수 합                   ([30경쟁][100대열차] +=  [30경쟁][100대열차][11개의 역])
                Score_per_train[k][i] += ScoreFunction(speedlist[k][i][j])


    # 하루마다의 점수 
    Score_per_all = [0 for i in range(N)]

    # 30개의 경쟁            
    for i in range(N):
        # 100대의 열차
        for j in range(One_day_train):
             Score_per_all[i] += Score_per_train[i][j]
                
    return Score_per_all

In [85]:
Score_per_all

[849588.0,
 846522.0,
 853110.0,
 872268.0,
 831588.0,
 850560.0,
 863238.0,
 869748.0,
 835362.0,
 885504.0,
 852996.0,
 862044.0,
 837168.0,
 860814.0,
 865092.0,
 841092.0,
 887442.0,
 883410.0,
 866310.0,
 865446.0,
 850338.0,
 844134.0,
 844908.0,
 861120.0,
 834510.0,
 875940.0,
 868206.0,
 862758.0,
 856350.0,
 851550.0]

In [90]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]), len(speedlist[0][0][0]),"총개수 : " ,len(speedlist) * len(speedlist[0]) * len(speedlist[0][0]) * len(speedlist[0][0][0])
)

print("-"*100)

#하루마다의 점수 (이걸로 경쟁)
print(Score_per_all[1])
#열차마다의 점수
print(Score_per_train[1][2])
#구간마다의 점수
print(ScoreFunction(speedlist[1][2][1]))


30 100 11 8 총개수 :  264000
----------------------------------------------------------------------------------------------------
846522.0
6417.0
1041.0


In [98]:
def Group_assessment(Score_per_all):
    # 적합도 비율 찾기
    speedlist_evaluation = [[Score_per_all[i], 0, i] for i in range(N)]

    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += Score_per_all[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][1] = speedlist_evaluation[i][0]/Sum_of_fitness
        else:
            speedlist_evaluation[i][1] = speedlist_evaluation[i-1][1]  + speedlist_evaluation[i][0]/Sum_of_fitness
    
    return speedlist_evaluation

#적합도 찾고 룰렛 준비
Group_assessment_value = Group_assessment(Score_per_all)

#상위 5개값                           정렬  점수값이 높은 상위 5개로 
sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
save_assessmend_value = sortand_Group_assessmend[0:save_value]
save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]

save_assessmend_num

#Score_per_all[16]

887442.0

In [30]:
def Group_assessment(Score_per_all):
    # 적합도 비율 찾기
    # (각자의 값+이전값 총합)/총합
    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += One_day_train_list[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][3] = speedlist_evaluation[i][2]/Sum_of_fitness
        else:
            speedlist_evaluation[i][3] = speedlist_evaluation[i-1][3]  + speedlist_evaluation[i][2]/Sum_of_fitness
    
    return speedlist_evaluation


Group_assessment_value = Group_assessment(speedlist)
#Group_assessment_value
sortand = sorted(Group_assessment_value, key=operator.itemgetter(2),reverse=True)
save_assessmend_value = sortand[0:save_value]
save_assessmend_num = [save_assessmend_value[i][4] for i in range(save_value)]
print(save_assessmend_num)

save_assessmend_value

264000